In [ ]:
!pip install unsloth
!pip install --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [ ]:
!pip install datasets


In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from unsloth import is_bfloat16_supported
from huggingface_hub import login
from transformers import TrainingArguments
from datasets import load_dataset
import wandb
from google.colab import userdata
userdata.get('Hugging_Face_Token')
userdata.get('wnb')

In [ ]:
HF_token=userdata.get('Hugging_Face_Token')
wnb_token=userdata.get('wnb')

login(HF_token)
wandb.login(key=wnb_token)
run=wandb.init(
    project="Fine tune DeepSeek-R1-Llama-8B on Medical COT Dataset",
    job_type="training",
    anonymous="allow"
)

In [ ]:
model,tokenizer=FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Llama-8B",
    max_seq_length=2048,
    dtype=None,
    load_in_4bit=True,
    token=HF_token
)

In [ ]:
prompt_style="""Below is an instruction that describes a task ,paired with an input that provides further context .
Write a response that appropriately completes the request.
Before answering,think carefully about the question and create a step by step chain-of-thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced medical knowledge in clinical reasoning,diagonstics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>{}
"""

In [ ]:
# Creating test medical question for inference
question="""A 61-year-old woman with a long history of involuntary urine loss during activities like coughing and sneezing but no leakage at night undergoes a gynelogical exam
         and Q-tip test.Based on this findings, what would csytometry most likely reveal about her residual volume and detrusor contractions? """

# Enabling optimized infernece mode for unsloth models
FastLanguageModel.for_inference(model)

# Format the question using structured prompt (prompt_style) and tokenize it.
inputs=tokenizer([prompt_style.format(question,"")],return_tensors='pt').to('cuda')

# Generate response using the model
outputs=model.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True
)

# Decode response into human readable text
response=tokenizer.batch_decode(outputs)



In [ ]:
# Extract and print only relevant response part(after ### Response)
print(response[0].split('### Response')[1])

In [ ]:
train_prompt_style="""Below is an instruction that describes a task ,paired with an input that provides further context .
Write a response that appropriately completes the request.
Before answering,think carefully about the question and create a step by step chain-of-thoughts to ensure a logical and accurate response.

### Instruction:
You are a medical expert with advanced medical knowledge in clinical reasoning,diagonstics, and treatment planning.
Please answer the following medical question.

### Question:
{}

### Response:
<think>
{}
</think>
{}"""

In [ ]:
# Download medical dataset from HuggingFace using load_dataset function
dataset=load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT","en",split="train[0:500]",trust_remote_code=True)
dataset

In [ ]:
dataset[1]

In [ ]:
# We need to format dataset according to our prompt training style
EOS_TOKEN=tokenizer.eos_token
EOS_TOKEN

In [ ]:
# Defining formatting prompt function
def formatting_prompt_func(examples):
  inputs=examples['Question']
  cots=examples['Complex_CoT']
  outputs=examples['Response']

  texts=[]

  for input,cot,output in zip(inputs,cots,outputs):
   text=train_prompt_style.format(input,cot,output) + EOS_TOKEN
   texts.append(text)

  return{"text":texts}


In [ ]:
# Update dataset formatting
dataset_finetune=dataset.map(formatting_prompt_func,batched=True)
dataset_finetune["text"][0]

In [ ]:
# Applying LoRA(Low Rank Adaption)fine tuning to the model
model_lora=FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None
)

In [ ]:
trainer=SFTTrainer(
    model=model_lora,
    tokenizer=tokenizer,
    train_dataset=dataset_finetune,
    dataset_text_field="text",
    max_seq_length=2048,
    dataset_num_proc=2,

    # Training arguments
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        num_train_epochs=1,
        warmup_steps=5,
        max_steps=60,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs"
    )
)

In [ ]:
# Start fine tuning process and saving the fine tuned model
trainer_stats=trainer.train()
wandb.finish()

In [ ]:
# Creating test medical question for inference
question="""A 61-year-old woman with a long history of involuntary urine loss during activities like coughing and sneezing but no leakage at night undergoes a gynelogical exam
         and Q-tip test.Based on this findings, what would csytometry most likely reveal about her residual volume and detrusor contractions? """

# Enabling optimized infernece mode for unsloth models
FastLanguageModel.for_inference(model_lora)

# Format the question using structured prompt (prompt_style) and tokenize it.
inputs=tokenizer([prompt_style.format(question,"")],return_tensors='pt').to('cuda')

# Generate response using the model
outputs=model_lora.generate(
    input_ids=inputs.input_ids,
    attention_mask=inputs.attention_mask,
    max_new_tokens=1200,
    use_cache=True
)

# Decode response into human readable text
response=tokenizer.batch_decode(outputs)

# Extract and print only relevant response part(after ### Response)
print(response[0].split('### Response:')[1])